In [1]:
import time
import math
import numpy as np
import pandas as pd
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

In [53]:
def data_preprocessing(data):
    data['date'] = list(map(lambda x, y: str(x) + '.' + str(y), data['regYear'], data['regMonth']))
    data['date'] = pd.to_datetime(data['date'])
    data['year'] = data['date'].apply(lambda x: x.year)
    data['month'] = data['date'].apply(lambda x: x.month)
    data['time_id'] = list(map(lambda x, y: (x-2016)*12 + y, data['year'], data['month']))
    data.drop(['regYear', 'regMonth', 'date'], axis=1, inplace=True)
    
    if 'forecastVolum' in list(data.columns):
        data.drop(['forecastVolum'], axis=1, inplace=True)
    if 'province' in list(data.columns):
        pro_label = dict(zip(sorted(list(set(data['province']))), range(0, len(set(data['province'])))))
        data['pro_id'] = data['province'].map(pro_label)
        data.drop(['province', 'adcode'], axis=1, inplace=True)
    if 'bodyType' in list(data.columns):
        body_label = dict(zip(sorted(list(set(data['bodyType']))), range(0, len(set(data['bodyType'])))))
        data['body_id'] = data['bodyType'].map(body_label)
        data = data.drop('bodyType', axis=1)
    model_label = dict(zip(sorted(list(set(data['model']))), range(0, len(set(data['model'])))))
    data['model_id'] = data['model'].map(model_label)
    data.drop('model', axis=1, inplace=True)
    
    return data


def feature_engineering(df, month):
    data = df.copy()
    data['jidu_id'] = ((data['month']-1)/3+1).map(int)
    features = []
    start = int((month-24)/3)*2
    start += int((month-24)/4)
    start = start-1 if start >=1 else start
    
    # history sales
    for last in range(1,17):  
        tmp=data.copy()
        tmp['time_id'] = tmp['time_id'].apply(lambda x: x+last+start if x+last+start<=28 else -1)
        tmp = tmp[tmp['time_id']!=-1][['label','time_id','pro_id','model_id','body_id']]
        tmp = tmp.rename(columns={'label':'last_{0}_sales'.format(last)})
        data = pd.merge(data,tmp,how='left',on=['time_id','pro_id','model_id','body_id'])
        if last <= 6:
            features.append('last_{0}_sales'.format(last)) 
    
    # history popularity
    for last in range(1,17):  
        tmp=data.copy()
        tmp['time_id'] = tmp['time_id'].apply(lambda x: x+last+start if x+last+start<=28 else -1)
        tmp=tmp[tmp['time_id']!=-1][['popularity','time_id','pro_id','model_id','body_id']]
        tmp=tmp.rename(columns={'popularity':'last_{0}_popularity'.format(last)})
        data=pd.merge(data,tmp,how='left',on=['time_id','pro_id','model_id','body_id'])
        if last<=6 or (last>=11 and last<=13):
            features.append('last_{0}_popularity'.format(last)) 

    # half year sales statistics
    data['half_year_sales_sum'] = data.loc[:,'last_1_sales':'last_6_sales'].sum(1)
    data['half_year_sales_mean'] = data.loc[:,'last_1_sales':'last_6_sales'].mean(1)
    data['half_year_sales_max'] = data.loc[:,'last_1_sales':'last_6_sales'].max(1)
    data['half_year_sales_min'] = data.loc[:,'last_1_sales':'last_6_sales'].min(1)
    data['1st_quarter_sales_sum']  = data.loc[:,'last_1_sales':'last_3_sales'].sum(1)
    data['2nd_quarter_sales_sum']  = data.loc[:,'last_4_sales':'last_6_sales'].sum(1)
    data['1st_quarter_sales_mean'] = data.loc[:,'last_1_sales':'last_3_sales'].mean(1)
    data['2nd_quarter_sales_mean'] = data.loc[:,'last_4_sales':'last_6_sales'].mean(1)
    sales_stat_feat = ['half_year_sales_sum','half_year_sales_mean','half_year_sales_max','half_year_sales_min',
                       '1st_quarter_sales_sum','2nd_quarter_sales_sum','1st_quarter_sales_mean','2nd_quarter_sales_mean']
    features = features + sales_stat_feat
    
    # trend
    data['1_2_diff'] = data['last_1_sales'] - data['last_2_sales']
    data['1_3_diff'] = data['last_1_sales'] - data['last_3_sales']
    data['2_3_diff'] = data['last_2_sales'] - data['last_3_sales']
    data['2_4_diff'] = data['last_2_sales'] - data['last_4_sales']
    data['3_4_diff'] = data['last_3_sales'] - data['last_4_sales']
    data['3_5_diff'] = data['last_3_sales'] - data['last_5_sales']
    data['quarter_1_2_diff'] = data['1st_quarter_sales_sum'] - data['2nd_quarter_sales_sum']
    trend_stat_feat = ['1_2_diff','1_3_diff','2_3_diff','2_4_diff','3_4_diff','3_5_diff','quarter_1_2_diff']
    features = features + trend_stat_feat

    # CNY
    data['is_CNY'] = list(map(lambda x:1 if x==2 or x==13 or x==26 else 0,data['time_id']))
    data['is_before_CNY'] = list(map(lambda x:1 if x==1 or x==12 or x==25 else 0,data['time_id']))
    data['is_after_CNY']   = list(map(lambda x:1 if x==3 or x==14 or x==27 else 0,data['time_id']))
    month_city_stat_feat = ['is_CNY','is_before_CNY','is_after_CNY']
    features = features + month_city_stat_feat
    
    # last two months' sales difference by model
    pivot = pd.pivot_table(data,index=['model_id'],values='1_2_diff',aggfunc=np.sum)
    pivot = pd.DataFrame(pivot).rename(columns={'1_2_diff':'model_1_2_diff_sum'}).reset_index()
    data  = pd.merge(data,pivot,on=['model_id'],how='left')
    
    # last two months' sales difference by province
    pivot = pd.pivot_table(data,index=['pro_id'],values='1_2_diff',aggfunc=np.sum)
    pivot = pd.DataFrame(pivot).rename(columns={'1_2_diff':'pro_1_2_diff_sum'}).reset_index()
    data  = pd.merge(data,pivot,on=['pro_id'],how='left')
    
    # last two months' sales difference by model and province
    pivot = pd.pivot_table(data,index=['pro_id','model_id'],values='1_2_diff',aggfunc=np.sum)
    pivot = pd.DataFrame(pivot).rename(columns={'1_2_diff':'model_pro_1_2_diff_sum'}).reset_index()
    data  = pd.merge(data,pivot,on=['pro_id','model_id'],how='left')
    pivot = pd.pivot_table(data,index=['pro_id','model_id'],values='1_2_diff',aggfunc=np.mean)
    pivot = pd.DataFrame(pivot).rename(columns={'1_2_diff':'model_pro_1_2_diff_mean'}).reset_index()
    data  = pd.merge(data,pivot,on=['pro_id','model_id'],how='left')
    two_month_stat_feat = ['model_1_2_diff_sum','pro_1_2_diff_sum','model_pro_1_2_diff_sum','model_pro_1_2_diff_mean']
    features = features + two_month_stat_feat
    
    # m/m growth
    data['mom_1_2'] = data['last_1_sales'] / data['last_2_sales']
    data['mom_2_3'] = data['last_2_sales'] / data['last_3_sales']
    data['mom_3_4'] = data['last_3_sales'] / data['last_4_sales']
    data['mom_4_5'] = data['last_4_sales'] / data['last_5_sales']
    data['mom_5_6'] = data['last_5_sales'] / data['last_6_sales']
    ring_ratio_stat_feat = ['mom_1_2','mom_2_3','mom_3_4','mom_4_5', 'mom_5_6']
    features = features + ring_ratio_stat_feat

    'm/m growth of m/m growth'
    data['mom_1_2_2_3'] = data['mom_1_2'] / data['mom_2_3']
    data['mom_2_3_3_4'] = data['mom_2_3'] / data['mom_3_4']
    data['mom_3_4_4_5'] = data['mom_3_4'] - data['mom_4_5']
    data['mom_4_5_5_6'] = data['mom_4_5'] - data['mom_5_6']
    two_ring_ratio_stat_feat = ['mom_1_2_2_3','mom_2_3_3_4','mom_3_4_4_5','mom_4_5_5_6']
    features = features + two_ring_ratio_stat_feat

    # sales by bodytype and province
    for i in range(1,7):
        last_time='last_{0}_sales'.format(i)
        pivot = pd.pivot_table(data,index=['time_id','pro_id','body_id'],values=last_time,aggfunc=np.sum)
        pivot = pd.DataFrame(pivot).rename(columns={last_time:'pro_body_last_{0}_sale_sum'.format(i)}).reset_index()
        data  = pd.merge(data,pivot,on=['time_id','pro_id','body_id'],how='left')
        data['last_{0}_sale_ratio_pro_body_last_{0}_sale_sum'.format(i,i)]=list(map(lambda x,y:x/y if y!=0 else 0,data[last_time],data['pro_body_last_{0}_sale_sum'.format(i)]))
        features.append('last_{0}_sale_ratio_pro_body_last_{0}_sale_sum'.format(i,i))
        if i>=2:
            data['last_{0}_{1}_sale_pro_body_diff'.format(i-1,i)] = data['last_{0}_sale_ratio_pro_body_last_{0}_sale_sum'.format(i-1)]-data['last_{0}_sale_ratio_pro_body_last_{0}_sale_sum'.format(i)]
            features.append('last_{0}_{1}_sale_pro_body_diff'.format(i-1,i))

    # sales by province
    for i in range(1,7):
        last_time = 'last_{0}_sales'.format(i)
        pivot = pd.pivot_table(data,index=['time_id','pro_id'],values=last_time,aggfunc=np.sum)
        pivot = pd.DataFrame(pivot).rename(columns={last_time:'pro__last_{0}_sale_sum'.format(i)}).reset_index()
        data  = pd.merge(data,pivot,on=['time_id','pro_id'],how='left')
        data['last_{0}_sale_ratio_pro_last_{0}_sale_sum'.format(i,i)]=list(map(lambda x,y:x/y if y!=0 else 0,data[last_time],data['pro__last_{0}_sale_sum'.format(i)]))
        features.append('last_{0}_sale_ratio_pro_last_{0}_sale_sum'.format(i,i))
        if i>=2:
            data['model_last_{0}_{1}_sale_pro_diff'.format(i-1,i)] = data['last_{0}_sale_ratio_pro_last_{0}_sale_sum'.format(i-1)]-data['last_{0}_sale_ratio_pro_last_{0}_sale_sum'.format(i)]
            features.append('model_last_{0}_{1}_sale_pro_diff'.format(i-1,i))

    # popularity m/m growth
    data['mom_1_2popularity'] = (data['last_1_popularity'] - data['last_2_popularity']) / data['last_2_popularity']
    data['mom_2_3popularity'] = (data['last_2_popularity'] - data['last_3_popularity']) / data['last_3_popularity']
    data['mom_3_4popularity'] = (data['last_3_popularity'] - data['last_4_popularity']) / data['last_4_popularity']
    data['mom_4_5popularity'] = (data['last_4_popularity'] - data['last_5_popularity']) / data['last_5_popularity']
    data['mom_5_6popularity'] = (data['last_5_popularity'] - data['last_6_popularity']) / data['last_6_popularity']
    popularity_ratio_stat_feat = ['mom_1_2popularity','mom_2_3popularity','mom_3_4popularity','mom_4_5popularity','mom_5_6popularity']
    features = features + popularity_ratio_stat_feat

    # popularity by model
    for i in range(1,7):
        last_time='last_{0}_popularity'.format(i)
        pivot = pd.pivot_table(data,index=['time_id','model_id'],values=last_time,aggfunc=np.sum)
        pivot = pd.DataFrame(pivot).rename(columns={last_time:'model__last_{0}_popularity_sum'.format(i)}).reset_index()
        data  = pd.merge(data,pivot,on=['time_id','model_id'],how='left')
        data['last_{0}_popularity_ratio_model_last_{0}_popularity_sum'.format(i,i)]=list(map(lambda x,y:x/y if y!=0 else 0,data[last_time],data['model__last_{0}_popularity_sum'.format(i)]))
        features.append('last_{0}_popularity_ratio_model_last_{0}_popularity_sum'.format(i,i))  

    # popularity by bodytype
    for i in range(1,7):
        last_time='last_{0}_popularity'.format(i)
        pivot = pd.pivot_table(data,index=['time_id','body_id'],values=last_time,aggfunc=np.sum)
        pivot = pd.DataFrame(pivot).rename(columns={last_time:'body_last_{0}_popularity_sum'.format(i)}).reset_index()
        data  = pd.merge(data,pivot,on=['time_id','body_id'],how='left')
        data['last_{0}_popularity_ratio_body_last_{0}_popularity_sum'.format(i,i)]=list(map(lambda x,y:x/y if y!=0 else 0,data[last_time],data['body_last_{0}_popularity_sum'.format(i)]))
        if i>=2:
            data['last_{0}_{1}_popularity_body_diff'.format(i-1,i)] = (data['last_{0}_popularity_ratio_body_last_{0}_popularity_sum'.format(i-1)]-data['last_{0}_popularity_ratio_body_last_{0}_popularity_sum'.format(i)])/data['last_{0}_popularity_ratio_body_last_{0}_popularity_sum'.format(i)]
            features.append('last_{0}_{1}_popularity_body_diff'.format(i-1,i)) 

    # sales y/y growth
    data["yoy_16_4"]=data["last_16_sales"] / data["last_4_sales"] -1
    data["yoy_15_3"]=data["last_15_sales"] / data["last_3_sales"] -1
    data["yoy_14_2"]=data["last_14_sales"] / data["last_2_sales"] -1
    data["yoy_13_1"]=data["last_13_sales"] / data["last_1_sales"] -1
    data["yoy_12_0"]=data["last_12_sales"] / data["label"] -1
    
    yoy_growth_features = ['yoy_16_4', 'yoy_15_3', 'yoy_14_2', 'yoy_13_1', 'yoy_12_0']
    features = features + yoy_growth_features
    
    # last 12 months by model
    pivot = pd.pivot_table(data,index=['time_id', 'model_id'], values='last_12_sales',aggfunc=np.mean)
    pivot = pd.DataFrame(pivot).rename(columns={'last_12_sales':'model_last_12_sales_mean'}).reset_index()
    data  = pd.merge(data,pivot,on=['time_id', 'model_id'],how="left")
    pivot = pd.pivot_table(data,index=['time_id', 'model_id'],values='last_12_sales',aggfunc=np.min)
    pivot = pd.DataFrame(pivot).rename(columns={'last_12_sales':'model_last_12_sales_min'}).reset_index()
    data  = pd.merge(data,pivot,on=['time_id', 'model_id'],how="left")
    pivot = pd.pivot_table(data,index=['time_id', 'model_id'],values='last_12_sales',aggfunc=np.sum)
    pivot = pd.DataFrame(pivot).rename(columns={'last_12_sales':'model_last_12_sales_sum'}).reset_index()
    data  = pd.merge(data,pivot,on=['time_id', 'model_id'],how="left")
    pivot = pd.pivot_table(data,index=['time_id', 'model_id'],values='last_12_sales',aggfunc=np.max)
    pivot = pd.DataFrame(pivot).rename(columns={'last_12_sales':'model_last_12_sales_max'}).reset_index()
    data  = pd.merge(data,pivot,on=['time_id', 'model_id'],how="left")
    pivot = pd.pivot_table(data,index=['time_id', 'model_id'],values='last_12_sales',aggfunc=np.median)
    pivot = pd.DataFrame(pivot).rename(columns={'last_12_sales':'model_last_12_sales_median'}).reset_index()
    data  = pd.merge(data,pivot,on=['time_id', 'model_id'],how="left")
    model_last_12_sales = ['model_last_12_sales_mean', 'model_last_12_sales_min', 'model_last_12_sales_sum', 
                           'model_last_12_sales_max', 'model_last_12_sales_median']
    features = features + model_last_12_sales
    
    # last 12 months by province
    pivot = pd.pivot_table(data,index=['time_id', 'pro_id'], values='last_12_sales', aggfunc=np.mean)
    pivot = pd.DataFrame(pivot).rename(columns={'last_12_sales':'pro_last_12_sales_mean'}).reset_index()
    data  = pd.merge(data,pivot,on=['time_id', 'pro_id'],how="left")
    pivot = pd.pivot_table(data,index=['time_id', 'pro_id'],values='last_12_sales', aggfunc=np.min)
    pivot = pd.DataFrame(pivot).rename(columns={'last_12_sales':'pro_last_12_sales_min'}).reset_index()
    data  = pd.merge(data,pivot,on=['time_id', 'pro_id'],how="left")
    pivot = pd.pivot_table(data,index=['time_id', 'pro_id'],values='last_12_sales', aggfunc=np.sum)
    pivot = pd.DataFrame(pivot).rename(columns={'last_12_sales':'pro_last_12_sales_sum'}).reset_index()
    data  = pd.merge(data,pivot,on=['time_id', 'pro_id'],how="left")
    pivot = pd.pivot_table(data,index=['time_id', 'pro_id'],values='last_12_sales', aggfunc=np.max)
    pivot = pd.DataFrame(pivot).rename(columns={'last_12_sales':'pro_last_12_sales_max'}).reset_index()
    data  = pd.merge(data,pivot,on=['time_id', 'pro_id'],how="left")
    pivot = pd.pivot_table(data,index=['time_id', 'pro_id'],values='last_12_sales', aggfunc=np.median)
    pivot = pd.DataFrame(pivot).rename(columns={'last_12_sales':'pro_last_12_sales_median'}).reset_index()
    data  = pd.merge(data,pivot,on=['time_id', 'pro_id'],how="left")
    province_last_12_sales = ['pro_last_12_sales_mean', 'pro_last_12_sales_min', 'pro_last_12_sales_sum', 
                           'pro_last_12_sales_max', 'pro_last_12_sales_median']
    features = features + province_last_12_sales
    
    
    
    # last 12 months by province, model
    pivot = pd.pivot_table(data,index=['time_id', 'pro_id', 'model_id'], values='last_12_sales',aggfunc=np.mean)
    pivot = pd.DataFrame(pivot).rename(columns={'last_12_sales':'pro_model_last_12_sales_mean'}).reset_index()
    data  = pd.merge(data,pivot,on=['time_id', 'pro_id', 'model_id'],how="left")
    pivot = pd.pivot_table(data,index=['time_id', 'pro_id', 'model_id'],values='last_12_sales',aggfunc=np.min)
    pivot = pd.DataFrame(pivot).rename(columns={'last_12_sales':'pro_model_last_12_sales_min'}).reset_index()
    data  = pd.merge(data,pivot,on=['time_id', 'pro_id', 'model_id'],how="left")
    pivot = pd.pivot_table(data,index=['time_id', 'pro_id', 'model_id'],values='last_12_sales',aggfunc=np.sum)
    pivot = pd.DataFrame(pivot).rename(columns={'last_12_sales':'pro_model_last_12_sales_sum'}).reset_index()
    data  = pd.merge(data,pivot,on=['time_id', 'pro_id', 'model_id'],how="left")
    pivot = pd.pivot_table(data,index=['time_id', 'pro_id', 'model_id'],values='last_12_sales',aggfunc=np.max)
    pivot = pd.DataFrame(pivot).rename(columns={'last_12_sales':'pro_model_last_12_sales_max'}).reset_index()
    data  = pd.merge(data,pivot,on=['time_id', 'pro_id', 'model_id'],how="left")
    pivot = pd.pivot_table(data,index=['time_id', 'pro_id', 'model_id'],values='last_12_sales',aggfunc=np.median)
    pivot = pd.DataFrame(pivot).rename(columns={'last_12_sales':'pro_model_last_12_sales_median'}).reset_index()
    data  = pd.merge(data,pivot,on=['time_id', 'pro_id', 'model_id'],how="left")
    pro_model_last_12_sales = ['pro_model_last_12_sales_mean', 'pro_model_last_12_sales_min', 'pro_model_last_12_sales_sum', 
                           'pro_model_last_12_sales_max', 'pro_model_last_12_sales_median']
    features = features + pro_model_last_12_sales
    
    
    # last 4 months sales by model
    for i in range(1,5):
        pivot = pd.pivot_table(data,index=["model_id","time_id"],values='last_{0}_sales'.format(i),aggfunc=np.mean)
        pivot = pd.DataFrame(pivot).rename(columns={'last_{0}_sales'.format(i):'model_last_{}_sales_mean'.format(i)}).reset_index()
        data  = pd.merge(data,pivot,on=["model_id","time_id"],how="left")
        pivot = pd.pivot_table(data,index=["model_id","time_id"],values='last_{0}_sales'.format(i+12),aggfunc=np.mean)
        pivot = pd.DataFrame(pivot).rename(columns={'last_{0}_sales'.format(i+12):'model_last_{}_sales_mean'.format(i+12)}).reset_index()
        data  = pd.merge(data,pivot,on=["model_id","time_id"],how="left")
        features.append('model_last_{}_sales_mean'.format(i))
        features.append('model_last_{}_sales_mean'.format(i+12))
    data["model_growth_14_2"] = data["model_last_14_sales_mean"] / data["model_last_2_sales_mean"] -1
    data["model_growth_13_1"] = data["model_last_13_sales_mean"] / data["model_last_1_sales_mean"] -1
    data["model_growth_15_3"] = data["model_last_15_sales_mean"] / data["model_last_3_sales_mean"] -1
    data["model_growth_16_4"] = data["model_last_16_sales_mean"] / data["model_last_4_sales_mean"] -1
    new_stat_feat = ["model_growth_14_2","model_growth_13_1","model_growth_15_3","model_growth_16_4"]
    
    return data, features

In [54]:
def train_model(data, features, month, model):
    df = data.copy()
        
    train_idx = df['time_id'].between(7, month-1)
    test_idx = df['time_id'].between(month, month)
    
    model.fit(df[train_idx][features], df[train_idx]['label'])
    df['forecastVolum'] = model.predict(df[features])
    sub = df[test_idx][['id']]
    sub['forecastVolum'] = df[test_idx]['forecastVolum']
    feat_importance = pd.DataFrame()
    feat_importance['feature'] = features
    feat_importance['score'] = model.feature_importances_
    return sub, feat_importance


def predict_by_month(data):
    feature_importance = pd.DataFrame()
    model = lgb.LGBMRegressor(
            num_leaves=2**5-1, reg_alpha=0.25, reg_lambda=0.25, objective='mse',
            max_depth=-1, learning_rate=0.05, min_child_samples=5, random_state=2019,
            n_estimators=600, subsample=0.9, colsample_bytree=0.7,
            )
    
    data['label'] = data['label'].apply(lambda x: x if x==np.NAN else math.log(x+1, 2))
    data['salesVolume'] = data['salesVolume'].apply(lambda x: x if x==np.NAN else math.log(x+1, 2))
    
    for month in [25, 26, 27, 28]:
        df, features = feature_engineering(data, month)
        cate_features = ['pro_id', 'model_id', 'body_id', 'year', 'month']
        for col in cate_features:
            df[col] = df[col].astype('category')
        all_features = features + cate_features
        sub, importance = train_model(df, all_features, month, model)
        feature_importance = pd.concat([feature_importance, importance], axis=1)
        data.loc[(data.time_id==month),  'salesVolume'] = sub['forecastVolum'].values
        data.loc[(data.time_id==month),  'label'] = sub['forecastVolum'].values

    data['salesVolume'] = data['salesVolume'].apply(lambda x: x if x==np.NAN else (2**(x))-1)
    sub = data.loc[(data.time_id >= 25),['id','salesVolume']]
    sub = sub.rename(columns={'salesVolume': 'forecastVolum'})
    sub['id'] = sub['id'].map(int)
    sub['forecastVolum'] = sub['forecastVolum'].map(round)
    return sub, feature_importance

In [55]:
if __name__ == '__main__':
    
    train_data = pd.read_csv('train_sales_data.csv')
    test_data = pd.read_csv('evaluation_public.csv')
    search_data = pd.read_csv('train_search_data.csv')
    
    train_data = data_preprocessing(train_data)
    test_data = data_preprocessing(test_data)
    search_data = data_preprocessing(search_data)
    
    # fill in bodytype for test data
    test_data['body_id'] = train_data['model_id'].map(train_data.drop_duplicates('model_id').set_index('model_id')['body_id'])

    data = pd.merge(train_data, search_data, how='left', on=['year', 'month', 'pro_id', 'model_id', 'time_id'])
    data = pd.concat([data, test_data])
    data['label'] = data['salesVolume']
    
    start = time.time()
    print('start training...')
    sub, feature_importance = predict_by_month(data)
    sub.to_csv('sub.csv', index=False)

start training...


In [57]:
feature_importance.to_csv('importance.csv')

In [58]:
sub

,id,forecastVolum
0,1,328
1,2,466
2,3,207
3,4,433
4,5,553
...,...,...
5275,5364,61
5276,5365,66
5277,5366,81
5278,5367,157
